<a href="https://colab.research.google.com/github/silverGriffin/CS114.K11/blob/master/MaskDetect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#sử dụng GPU của gg colab
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 145674 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.15-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.15-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.15-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [7]:
#tạo thư mục ảo tên "drive0" trên GG colab
!mkdir -p drive0
!google-drive-ocamlfuse drive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [3]:
# Device Kiểm tra thông số
import tensorflow as tf
from tensorflow.python.client import device_lib
print('Current Device: ', tf.test.gpu_device_name() + '\n')
device_lib.list_local_devices()

Current Device:  /device:GPU:0



[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 16884516841898861148, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 6816074787139539230
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 3766835733080479501
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 15956161332
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 6638057709229971266
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"]

In [4]:
# RAM: 
!cat /proc/meminfo

MemTotal:       13335188 kB
MemFree:         9966420 kB
MemAvailable:   12231028 kB
Buffers:           85016 kB
Cached:          2272288 kB
SwapCached:            0 kB
Active:           904632 kB
Inactive:        2037676 kB
Active(anon):     500804 kB
Inactive(anon):     8516 kB
Active(file):     403828 kB
Inactive(file):  2029160 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:               264 kB
Writeback:             0 kB
AnonPages:        585040 kB
Mapped:           467672 kB
Shmem:              9088 kB
Slab:             185028 kB
SReclaimable:     144592 kB
SUnreclaim:        40436 kB
KernelStack:        3844 kB
PageTables:         7080 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6667592 kB
Committed_AS:    3135572 kB
VmallocTotal:   34359738367 kB
VmallocUsed:           0 kB
VmallocChunk:          0 kB
AnonHugePages:         0 kB
ShmemHugePages:  

In [5]:
# CPU: 
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2200.000
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs
bogomips	: 4400.00
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 bits 

In [6]:
# Tensorflow version
!pip show tensorflow

Name: tensorflow
Version: 1.15.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: wheel, tensorboard, google-pasta, keras-applications, six, tensorflow-estimator, astor, wrapt, protobuf, absl-py, gast, termcolor, grpcio, numpy, keras-preprocessing, opt-einsum
Required-by: stable-baselines, magenta, fancyimpute


In [0]:
#Làm việc với thư mục: ví dụ trong thư mục drive0 có chưa thư mục drive0_demo
!ls drive0/drive0_demo #Liệt kê các file trong thư mục drive0_demo

import os
os.getcwd() #thư mục hiện tại
os.chdir("drive0") #truy cập đến thư mục drive0 
os.chdir("..") #quay về thư mục trước đó

In [0]:
#Cài đặt thư viện cần thiết: sử dụng 2 cú pháp "!pip install" và "!apt-get"
!pip install -q keras #keras
!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python #OpenCV
#import cv2

In [0]:
#Tải file hay thư mục
#tải CSV
#!wget https://raw.githubusercontent.com/vincentarelbundock/Rdatasets/master/csv/datasets/Titanic.csv -P drive/app
#tải thư mục git
#!os.chdir("drive0/drive0_demo")
#!git clone https://github.com/wxs/keras-mnist-tutorial.git

**Chạy file .ipynb hay.py:**
Với file .ipynb, ta mở trực tiếp trong GG Drive bằng GG Colab. Rồi chạy chương trình Nếu đã chạy câu lệnh git clone phía trên, ta có thể mở file MNIST in keras.ipynb ra để chạy thử. Ta sẽ thấy tốc độ chạy hơn CPU rất nhiều. Thông thường 1 epoch, batch size 128, mất khoảng 3-4s.

Với file .py, ta có thể tải file mnist_cnn.py hoặc upload file của mình lên GG Drive

os.getcwd() # Kiem tra thu muc hien tai

!python3 drive/app/mnist_cnn.py # Chay file mnist_cnn

In [8]:
#final
import cv2 #import các thư viện 
import os
import math
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
from google.colab import files
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from imutils import paths
from keras.applications import VGG16
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img

Using TensorFlow backend.


In [9]:
#final
from google.colab import drive #kết nối với gg drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

In [0]:
#final
def Get_first_face(image_path): #trả về khuôn mặt đầu tiên từ một bức ảnh và số lượng face trong 1 ảnh
  img = cv2.imread(image_path)
  cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
  faces = cascade.detectMultiScale(img)
  print('found', len(faces), 'faces', sep=' ')
  if len(faces) == 0: #không tìm thấy face nào trong ảnh
    return None, 0

  (x, y, w, h) = faces[0] #giả sử detect nhiều face thì lấy face đầu tiên 
  r = max(w, h) / 2
  centerx = x + w / 2
  centery = y + h / 2
  nx = int(centerx - r)
  ny = int(centery - r)
  nr = int(r * 2)

  faceimg = img[ny:ny+nr, nx:nx+nr]
  #faceimg = cv2.resize(faceimg,(50,50))
  #faceimg = cv2.cvtColor(faceimg, cv2.COLOR_BGR2GRAY)   #để dành làm ở bước sau
  return faceimg, len(faces) #độ dài mảng faces -> số lượng face detect


In [0]:
def Resize_image(faceimg): #resize ảnh (face) về kích thước 50x50 pixels, chuyển về màu xám
  faceimg=cv2.resize(faceimg, (50,50))
  faceimg=cv2.cvtColor(faceimg, cv2.COLOR_BGR2GRAY)
  return faceimg


In [12]:
#Tạo 2 pandas series X và Y từ thư mục chứa ảnh train
#Duyệt từng file trong thư mục, crop ảnh, resize ảnh crop về kích thước 50x50 và đổi sang ảnh xám. Sau đó flatten, và đưa vào X series. 
#Nếu ảnh không crop được cái face nào thì cho mặc định một face toàn màu đen. 
#Series Y được xác định dựa vào tên file ảnh. Tên bắt đầu bằng chữ 'pos' là có đeo khẩu trang, 'neg' là không đeo khẩu trang.

i = 1; #biến đếm i để chặn số lượng ảnh duyệt
#folder = '/mydrive/My Drive/Teaching/2019.hk2.cs114.mask.dataset/train'
work_folder = '/content/drive/My Drive/train'

all_files = os.listdir(work_folder)

#count = 1 -> i
face_amount = 0
dectect = 0
faces_in_image = 0

X = pd.DataFrame(columns=range(2500)) #50x50=2500
Y = pd.DataFrame(columns=['label']) #gán nhãn dựa vào tên bắt đầu của ảnh

for image in os.listdir(work_folder):
  print('====  process image', image, ':', i, 'out of', len(all_files), sep=' ')
  faceimg,faces_in_image = Get_first_face(work_folder + "/" + image)
  if faceimg is not None:
    #new_img = cv2.cvtColor(cv2.resize(faceimg,(50,50)), cv2.COLOR_BGR2GRAY)
    new_img=Resize_image(faceimg)

    flat = new_img.flatten()
    t = pd.DataFrame([flat])
    dectect += 1
    #print(t)

  else:
    print('Cant find face')
    t = pd.DataFrame([[0]*2500], columns=range(2500)) #màu đen

  X=X.append(t, ignore_index=True)

  Y=Y.append({'label':image[0:3]},ignore_index=True) #nhãn là pos hoặc neg

  i = i + 1
  """if i == 15: #chỉ duyệt qua 14 ảnh
    break;"""
print(X)
print(Y)

====  process image neg.18520245-1.jpg : 1 out of 218
found 3 faces
====  process image pos.18520285-2.jpg : 2 out of 218
found 2 faces
====  process image neg.18520339-11.jpg : 3 out of 218
found 6 faces
====  process image pos.18520590-2.jpg : 4 out of 218
found 2 faces
====  process image pos.17520374-8.jpg : 5 out of 218
found 9 faces
====  process image neg.16521206-19.jpg : 6 out of 218
found 1 faces
====  process image neg.18520245-10.jpg : 7 out of 218
found 2 faces
====  process image neg.18520285-12.jpg : 8 out of 218
found 1 faces
====  process image neg.18520245-8.jpg : 9 out of 218
found 2 faces
====  process image pos.18520184-9.jpg : 10 out of 218
found 8 faces
====  process image neg.18520590-7.jpg : 11 out of 218
found 1 faces
====  process image neg.18520339-10.jpg : 12 out of 218
found 4 faces
====  process image pos.18520184-4.jpg : 13 out of 218
found 0 faces
Cant find face
====  process image pos.18520184-5.jpg : 14 out of 218
found 0 faces
Cant find face
====  pr

In [13]:
Transform_to_bool = LabelEncoder()
labels = Transform_to_bool.fit_transform(Y) #chuyển nhãn pos thành 1, neg thành 0 để tiện xử lý
labels

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0,
       1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1,
       1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1])

In [14]:
#chọn model để học 
#chọn kiến trúc mạng CNN vgg16
#base_model = VGG16(weights = 'imagenet', include_top = False)
base_model = VGG16(include_top= False, weights='imagenet' , input_shape = (50,50,3))





58892288/58889256 [==============================] - 2s 0us/step








In [15]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 50, 50, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 50, 50, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 50, 50, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 25, 25, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 25, 25, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 25, 25, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0     

In [0]:
list_image = []

In [0]:
#trích xuất các features
features = model.predict(list_image)
# Chuyển tensor thành vector, convolution layer cuối (512*7*7)
features = features.reshape((features.shape[0], 512*7*7))

In [0]:
#chia tập dữ liệu ban đầu thành training_set và test_set với tỷ lệ 85:15

In [0]:
#đánh giá model

In [16]:
#Choose model and learn
from sklearn.svm import SVC

model = SVC()
model.fit( X,  Y.values.ravel()  )

print(model)



SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [17]:
#dùng pickle để lưu lại model
import pickle

string = pickle.dumps(model)
print(string)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [0]:
def show_metrics(X, Y, string):
  import pickle
  print(Y.values.ravel())

  loadmodel = pickle.loads(string)
  pred = loadmodel.predict(X)
  print(pred)
  from sklearn import metrics
  print("Acurracy ", metrics.accuracy_score(Y.values.ravel(), pred))
  print("Precision ", metrics.precision_score(Y.values.ravel(), pred, pos_label='pos'))
  print("Recall ", metrics.recall_score(Y.values.ravel(), pred, pos_label='pos'))
  print("F1 ", metrics.f1_score(Y.values.ravel(), pred, pos_label='pos'))
  #print("Acurracy ", metrics.accuracy_score(Y_test.values.ravel(), pred))